In [ ]:
# Install dependencies (run this cell first!)
!uv pip install -q "pydantic-ai-slim[logfire,openai,tavily,a2a]>=0.0.49" "pydantic-settings>=2.0.0" "logfire[httpx]>=4.3.3" "httpx>=0.27.0"
print("✅ Dependencies installed successfully!")

# Lecture 05: Agent-to-Agent (A2A) Protocol Communication

This notebook demonstrates the **Agent-to-Agent (A2A) protocol** - an open standard introduced by Google that enables communication between AI agents over HTTP.

**⚠️ IMPORTANT**: Before running this notebook, you must start the A2A stock analysis server in a separate terminal:

```bash
uvicorn lecture05.stock_a2a_server:app --host 0.0.0.0 --port 8001
```

**Reference**: [Pydantic AI A2A Documentation](https://ai.pydantic.dev/a2a/)


In [ ]:
# Setup: Import necessary modules
from pydantic_ai import Agent
from common.tools import web_search_tool
from common.utils import create_agent_model, setup_logging
from lecture05.agent import a2a_stock_analysis_tool  # Pre-built A2A tool

# Initialize logging
setup_logging()

In [ ]:
# Add Logfire setup
from common.utils import setup_logfire

setup_logfire(
    service_name="a2a-protocol-lecture05",
    start_message="🚀 Lecture 05 - A2A Protocol Notebook Started",
)

In [ ]:
# Create financial assistant with A2A communication capability
# The a2a_stock_analysis_tool communicates with a stock analysis agent
# running as a separate A2A server over HTTP
a2a_financial_agent = Agent(
    model=create_agent_model(),
    tools=[web_search_tool, a2a_stock_analysis_tool],
    system_prompt="""
You are a knowledgeable financial assistant with A2A protocol communication.

When answering questions:
1. Use web_search for general financial information
2. Use request_stock_analysis_a2a to communicate with the stock analysis agent via A2A protocol
   - This sends HTTP requests to localhost:8001 using the A2A standard
   - The stock analysis agent runs as a separate A2A server
3. Always mention when you're using the A2A protocol

Keep your responses informative and concise.
""",
)

In [ ]:
# Ask a general financial question (uses web_search)
question = "What is the current price of Bitcoin?"
result = await a2a_financial_agent.run(question)
print(f"Question: {question}\n")
print(f"Response:\n{result.output}")

In [ ]:
# Ask for detailed stock analysis (uses A2A protocol)
# This will:
# 1. Send HTTP request to A2A server at localhost:8001
# 2. Use A2A protocol message format
# 3. Receive StockReport from A2A server
question = "Can you give me a detailed analysis of Apple stock (AAPL)?"
result = await a2a_financial_agent.run(question)
print(f"Question: {question}\n")
print(f"Response:\n{result.output}")

In [ ]:
# Try another stock analysis via A2A protocol
question = "Give me a comprehensive report on Tesla (TSLA)"
result = await a2a_financial_agent.run(question)
print(f"Question: {question}\n")
print(f"Response:\n{result.output}")

## Comparison: Agent-as-Tool vs. A2A Protocol

| Aspect | Lecture 03 (Agent-as-Tool) | Lecture 05 (A2A Protocol) |
|--------|---------------------------|---------------------------|
| Pattern | Direct code integration | HTTP protocol communication |
| Communication | In-process function call | HTTP requests to A2A server |
| Protocol | None | Google's A2A standard |
| Agent Location | Same process | Separate service/server |
| Interoperability | Framework-specific | Cross-framework/vendor |
| Deployment | Single application | Distributed services |
| Scalability | Limited to process | Horizontally scalable |
